In [1]:
from opal_ptx import CuModuleWrapper, kernel_transformer
from torch.utils.cpp_extension import load
import torch

from matplotlib import pyplot as plt

In [2]:
@kernel_transformer.fragment()
def warp_sum(out: "u32", a: "u32"):
    accum: u32 = a 
    d: u32
    i = 16
    while i > 0:
        ptx.shfl.sync.down.b32(d, accum, i, 31, 0xFFFFFFFF) 
        accum += d
        i //= 2
    if u32("%laneid") == 0:
        out = accum

@kernel_transformer.kernel()
def shfl_test(buffer: "u64"):
    a: u32 = u32("%laneid")
    result: u32
    warp_sum(kernel_builder, result, a)
    
    
    buffer_global: u64
    
    tidx: u32 = u32("%tid.x") + u32("%ntid.x") * u32("%ctaid.x")
    buffer += tidx * 4
    
    ptx.cvta.to._global.u64(buffer_global, buffer)
    if u32("%laneid") == 0:
        ptx.st._global.u32([buffer_global], result)

In [3]:
kernel_builder = kernel_transformer.KernelBuilder()
shfl_test(kernel_builder)
kernel_code = kernel_builder.generate("sm_75")

result = torch.zeros((64,), dtype=torch.uint32, device="cuda")

wrapper = CuModuleWrapper()
wrapper.load_ptx_code(kernel_code)

wrapper.launch_kernel("shfl_test", (1, 1, 1), (64, 1, 1), (result.data_ptr(),), 0)
print(result.cpu())

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint32)
